In [ ]:
from pathlib import Path
import sys
#src_path = str(Path.cwd().parent / "src")
#sys.path.append(src_path)

from Supply_chain_disruption_model import SimulationModel, generate_data, PN_score

import numpy as np
#from scipy.stats import poisson
#import random
import pandas as pd
import plotly.express as px

#!pip install scikit-survival

from datetime import datetime

In [ ]:
adj, C, sector = generate_data(dim=100, nb_s=3)
A = np.copy(adj)
A[A <= 0] = 0
print(f"{np.sum(A > 0)} non-zero entries, which is {np.sum(A > 0)*100/50**2 :.2f}%")

# Simulation

Define model parameters.

In [ ]:
# number of firms
dim = 100
# percentage of firms which are damaged / 100
p = 0.2
damage_level = 0.5
margin = 0.2
# time-period over which inventory is restored (days)
tau = 6
# average nb days of product each firm has in inventory for target inventory value
# value=9 from paper inoue_firm-level_2019
k = 9
# number of sectors
nb_s = 3
# recovery rate
gamma = 0.2
# number of days without recovery
sigma = 6
alpha = 3
u = 0.8

nb_iter = 1000

Generate data.

In [ ]:
adj, C, sector = generate_data(dim, nb_s)
A = np.copy(adj)
A[A <= 0] = 0

Execute the simulation.

In [ ]:
# intialize the model
mdl = SimulationModel(A, sector, C, p, damage_level, margin, k, gamma, tau, sigma, alpha, u, nb_iter)

# time the simulation
start=datetime.now()  

# run the model
mdl.run_simulation(print_iter=False)

print(f"runtime: {datetime.now()-start}")
# for 1000 firms and 1000 iterations, runtime is 9 minutes

For the following setup, the runtime was `0:08:52.834090`:
 - `dim = 1000`
 - `p = 0.1`
 - `damage_level = 0.5`
 - `margin = 0.2`
 - `tau = 6`
 - `k = 9`
 - `nb_s = 25`
 - `gamma = 0.2`
 - `sigma = 6`
 - `alpha = 2`
 - `u = 0.8`
 - `nb_iter = 1000`

In [ ]:
print(f"{len(mdl.damaged_ind)} firms were damaged by the disruption, which is {100*len(mdl.damaged_ind)/dim :.1f}%.")
print(f"{len(mdl.defaults)} out of {dim} firms defaulted, which is {100*len(mdl.defaults)/dim :.1f}%.")

damaged_and_defaulted = list(set(mdl.damaged_ind) & set(mdl.defaults.keys()))
perc_damaged_and_defaulted_of_damaged = 100*len(damaged_and_defaulted)/len(mdl.damaged_ind)
print(f"Of the damaged firms, {perc_damaged_and_defaulted_of_damaged :.1f}% defaulted, "
      f"{100 - perc_damaged_and_defaulted_of_damaged :.1f}% survived.")

perc_damaged_and_defaulted_of_defaulted = 100*len(damaged_and_defaulted)/len(mdl.defaults)
print(f"Of the defaulted firms, {perc_damaged_and_defaulted_of_defaulted :.1f}% had been damaged, "
      f"{100 - perc_damaged_and_defaulted_of_defaulted :.1f}% had not been damaged. \n")

print(f"This means {100 - perc_damaged_and_defaulted_of_defaulted :.1f}% of the firms that defaulted did so due to "
      f"network propagation of the damage.")

#print(f"damaged firms that also defaulted: \n{sorted(damaged_and_defaulted)}\n")
#print(f"damaged (ind): \n{mdl.damaged_ind} \n")
#print(f"defaults (ind: iteration): \n{mdl.defaults} \n")
#print(f"defaulted firms: \n{sorted(mdl.defaults.keys())} \n")
#print(f"default times: \n{sorted(mdl.defaults.values())}")

In [ ]:
mdl.plot_act_capacity(col_by_sector=False)

In [ ]:
mdl.plot_rel_capacity(col_by_sector=False)

In [ ]:
mdl.plot_rel_capacity(col_by_sector=True)

# PN score

In [ ]:
scores = PN_score(adj).reshape(1,-1)[0]

In [ ]:
default_times = np.zeros(dim)
max_default_time = np.max(list(mdl.defaults.values()))
for i in range(len(default_times)):
    if i in mdl.defaults:
        default_times[i] = mdl.defaults[i] 
    else:
        default_times[i] = max_default_time + 10

In [ ]:
damaged = np.zeros(dim).astype(int).astype(str)
damaged[mdl.damaged_ind] = str(1)

defaulted = np.zeros(dim).astype(int).astype(str)
defaulted[list(mdl.defaults.keys())] = str(1)

In [ ]:
fig = px.scatter(x=scores, y=default_times, color=damaged, symbol=defaulted, 
                 title="Absolute values in adjacency matrix",
                 labels={'x':'PN score', 'y':'default time (day)', 'color':'damaged', 'symbol':'defaulted'}) 
# , trendline='ols'
fig.show()

In [ ]:
adj_bin = np.copy(adj)
adj_bin[adj_bin > 0] = 1
adj_bin[adj_bin < 0] = -1

scores_bin = PN_score(adj_bin).reshape(1,-1)[0]

In [ ]:
fig = px.scatter(x=scores_bin, y=default_times, color=damaged, symbol=defaulted, 
                 title="Binary values in adjacency matrix",
                 labels={'x':'PN score', 'y':'default time (day)', 'color':'damaged', 'symbol':'defaulted'}) 
# , trendline='ols'
fig.show()

# Survival analysis

In [ ]:
damaged_bool = damaged.astype(bool)
defaulted_bool = defaulted.astype(bool)

In [ ]:
data_y = pd.DataFrame(np.transpose([defaulted_bool,default_times]), columns=["defaulted","default_time"])
data_y["defaulted"] = data_y["defaulted"].astype(bool)
data_y.head()

In [ ]:
data_x = pd.DataFrame(np.transpose([scores,damaged_bool]), columns=["PN","damaged"])
data_x["damaged"] = data_x["damaged"].astype(bool)
data_x.head()

In [ ]:
import matplotlib.pyplot as plt
from sksurv.nonparametric import kaplan_meier_estimator

time, survival_prob = kaplan_meier_estimator(data_y["defaulted"], data_y["default_time"])
plt.step(time, survival_prob, where="post")
plt.ylabel("est. probability of survival $\hat{S}(t)$")
plt.xlabel("time $t$")
plt.show()

In [ ]:
for damage_status in (True, False):
    mask_damaged = data_x["damaged"] == damage_status
    time_damaged, survival_prob_damaged = kaplan_meier_estimator(
        data_y["defaulted"][mask_damaged],
        data_y["default_time"][mask_damaged])

    plt.step(time_damaged, survival_prob_damaged, where="post",
             label="Damaged = %s" % damage_status)

plt.ylabel("est. probability of survival $\hat{S}(t)$")
plt.xlabel("time $t$")
plt.legend(loc="best")
plt.show()

In [ ]:
from sklearn import set_config
from sksurv.linear_model import CoxPHSurvivalAnalysis

set_config(display="text")  # displays text representation of estimators

data_x_numeric = data_x.copy()
data_x_numeric["damaged"] = data_x_numeric["damaged"].astype(float)

dt = np.dtype([('defaulted', '?'), ('default_time', '<f8')])
data_y = np.array(list(zip(defaulted_bool,default_times)), dtype=dt)

estimator = CoxPHSurvivalAnalysis()
estimator.fit(data_x_numeric, data_y)

In [ ]:
pd.Series(estimator.coef_, index=data_x_numeric.columns)

In [ ]:
estimator.score(data_x_numeric, data_y)